# Configure the SFT Model with LoRA

参考： https://docs.unsloth.ai/

In [1]:
from unsloth import FastModel
import torch
torch.cuda.empty_cache() 

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 08-22 12:56:57 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 08-22 12:56:57 [__init__.py:239] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
from datasets import load_dataset
from unsloth.chat_templates import get_chat_template
from unsloth.chat_templates import standardize_data_formats
from transformers import TextStreamer


In [3]:
ISTRAIN = True
MODEL_2_LOCAL= False

unsloth 支持的模型列表
https://docs.unsloth.ai/get-started/all-our-models

In [4]:

model_name = "unsloth/Qwen3-4B-Instruct-2507" # unsloth/ 前缀模型是预先优化的版本
# model_name="Qwen/Qwen3-0.6B"
local_save_path = "./unsloth_local_models/"+model_name
base_model, tokenizer =FastModel.from_pretrained(
    model_name = model_name,
    max_seq_length = 2048,
    dtype = None, #用模型默认
    load_in_4bit =True, # 模型的权重会以 4 位精度存储在内存中（节省空间） QLoRA or LoRA
    full_finetuning =False
    )

if MODEL_2_LOCAL:
    # 保存模型和分词器到本地
    base_model.save_pretrained(local_save_path)
    tokenizer.save_pretrained(local_save_path)

print("模型精度:", base_model.dtype)


==((====))==  Unsloth 2025.8.8: Fast Qwen3 patching. Transformers: 4.54.1. vLLM: 0.8.5.post1.
   \\   /|    NVIDIA GeForce RTX 3070 Ti Laptop GPU. Num GPUs = 1. Max memory: 8.0 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
模型精度: torch.bfloat16


Qwen-3 renders multi turn conversations like below:

```
<|im_start|>user
Hello!<|im_end|>
<|im_start|>assistant
Hey there!<|im_end|>

```

In [5]:
# 加了这个 enable_thinking=False 会失效
if model_name == "unsloth/Qwen3-4B-Instruct-2507":
    tokenizer = get_chat_template(
        tokenizer,
        chat_template = "qwen3-instruct",
    )


def gen_result(model,tokenzier,prompt):
    messages = [
        {"role" : "user", "content" : f"{prompt}"}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize = False,
        add_generation_prompt = True, # Must add for generation
        enable_thinking=False
    )
    model_inputs =tokenizer([text], return_tensors = "pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens = 512, # Increase for longer outputs!
        temperature = 0.7, top_p = 0.8, top_k = 20, # For non thinking
        streamer = TextStreamer(tokenizer, skip_prompt = True),
    )
    # 输出生成内容
    # output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()
    # content = tokenizer.decode(output_ids[0:], skip_special_tokens=True).strip("\n")
    # print("content:", content)



In [6]:
gen_result(base_model,tokenizer,"你是")

<|im_start|>user
你是<|im_end|>
<|im_start|>assistant
你好！我是Qwen，是阿里云研发的超大规模语言模型。我可以帮助你回答问题、创作文字，比如写故事、写公文、写邮件、写剧本等等，还能进行逻辑推理、编程等任务。如果你有任何问题或需要帮助，随时告诉我哦！😊<|im_end|>


问模型是谁，模型回答包含：Qwen 阿里云 ...


In [7]:
gen_result(base_model,tokenizer,"赫敏是谁")

赫敏（Hermione Granger）是英国作家J.K. 罗琳（J.K. Rowling）创作的奇幻小说《哈利·波特》系列中的一个核心角色。

她是主角哈利·波特和罗恩·韦斯莱的好友，也是三人组“哈利、罗恩、赫敏”中最具智慧和勇气的成员之一。

赫敏的基本信息如下：

- **全名**：赫敏·格兰杰（Hermione Granger）
- **出生年份**：1980年（在故事中）
- **出生地**：英国
- **家庭背景**：出身于一个普通巫师家庭，父母是普通巫师，母亲是“格兰杰夫人”，父亲是“格兰杰先生”，家庭并不富裕，但非常重视教育。
- **性格特点**：聪明、勤奋、勇敢、有责任感，同时也有点固执和强烈正义感。她非常热爱学习，尤其擅长魔法理论、变形术、魔药学等。

赫敏的突出特点包括：

1. **极高的智力**：她以“书呆子”著称，是霍格沃茨魔法学校中魔法知识最渊博的学生之一，尤其在魔药学、变形术和魔法史方面表现出色。

2. **忠诚与勇敢**：她始终站在哈利和罗恩身边，即使面对危险也从不退缩。在对抗伏地魔的过程中，她多次表现出非凡的勇气和领导力。

3. **成长与转变**：从一个害羞、内向的少女，成长为一个坚强、自信、有担当的巫师，甚至在后期担任了“魔法部”和“邓布利多军”（Dumbledore’s Army）的重要角色。

4. **对教育的热爱**：她非常重视知识和学习，也经常帮助其他学生，甚至在霍格沃茨中组织“魔法学习小组”。

5. **情感发展**：在系列后期，她与哈利和罗恩的关系逐渐深化，也展现了她对友情、责任和爱的理解。

赫敏是《哈利·波特》系列中最具智慧和成长弧线的角色之一，她不仅代表了“知识的力量”，也象征着普通人在面对不公与邪恶时，如何通过勇气和智慧去改变命运。

总结：  
👉 赫敏是《哈利·波特》中一位聪明、勇敢、善良、热爱学习的女巫，是


### Load the PEFT model and configure LoRA
target_modules:
* q_proj：将输入特征投影到查询（Query） 空间
* k_proj：将输入特征投影到键（Key） 空间
* v_proj：将输入特征投影到值（Value） 空间
* o_proj：将注意力计算的输出投影到最终的特征空间（Output Projection）
* up_proj：FFN 将特征维度升高（如从隐藏层维度升到 4 倍）
* down_proj：FFN 将升高后的维度降回原始隐藏层维度
* gate_proj: 在一些模型（如 Mistral）的 FeedForward 层中，用于计算门控机制的投影

In [8]:
#适配器
if ISTRAIN:
    peft_model = FastModel.get_peft_model( 
        base_model,
        target_modules=["q_proj","k_proj","v_proj","o_proj"],
        use_gradient_checkpointing= 'unsloth',
        r=16,  #LoRA 的秩（Rank），控制低秩矩阵的维度
        lora_alpha= 4, #LoRA 的缩放因子，用于调整低秩矩阵输出的权重
        lora_dropout=0,
        bias="none", #"none"：不训练任何偏置参数（最常用，节省计算），
        random_state= 42,
        use_rslora=False,
        loftq_config= None,
    )
    peft_model.print_trainable_parameters()

Unsloth: Making `model.base_model.model.model` require gradients
trainable params: 11,796,480 || all params: 4,034,264,576 || trainable%: 0.2924


### Load Finetome data

In [9]:
# data_name = "mlabonne/FineTome-100k"
# non_reasoning_data = load_dataset(data_name,split="train") 

# print(non_reasoning_data[1])
# print('---------------------------------')
# dataset = standardize_data_formats(non_reasoning_data)
# dataset[1]

# def formatting_prompts_func(examples):
#    convos = examples["conversations"]
#    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
#    return { "text" : texts, }

# dataset = dataset.map(formatting_prompts_func, batched = True)
# dataset[1]

## Load custom datas

In [ ]:
data_path = "./datasets/custom_data.json"
custom_data = load_dataset("json", data_files=data_path,split="train") 
print(custom_data[1])
print('-'*100)
dataset = standardize_data_formats(custom_data)
dataset[1]

def formatting_prompts_func(examples):

   convos = examples["conversations"]
   for conv in convos:
     if conv[1]["role"] == "assistant":
         conv[1]["content"] = conv[1]["content"].replace("我", "我<哈哈哈>") # add tag 

   texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
   return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True)
dataset[1]

{'conversations': [{'content': '你是谁', 'role': 'user'}, {'content': '我是为提供高效帮助而生的语言模型，可协助完成问答、创作、信息整理等任务，需要什么帮助吗？', 'role': 'assistant'}]}
----------------------------------------------------------------------------------------------------


{'conversations': [{'content': '你是谁', 'role': 'user'},
  {'content': '我<哈哈哈>是为提供高效帮助而生的语言模型，可协助完成问答、创作、信息整理等任务，需要什么帮助吗？',
   'role': 'assistant'}],
 'text': '<|im_start|>user\n你是谁<|im_end|>\n<|im_start|>assistant\n我<哈哈哈>是为提供高效帮助而生的语言模型，可协助完成问答、创作、信息整理等任务，需要什么帮助吗？<|im_end|>\n'}

### Import SFTTrainer and SFTConfig

In [11]:
from trl import SFTTrainer, SFTConfig
import trl
print(trl.__version__)
if ISTRAIN:
    trainer = SFTTrainer(
        model= peft_model,
        tokenizer = tokenizer,
        train_dataset= dataset,
        args =SFTConfig(
            dataset_text_field = "text",
            per_device_train_batch_size = 2,
            gradient_accumulation_steps = 4,
            learning_rate = 8e-5,
            weight_decay= 0.01,
            num_train_epochs = 3,
            fp16 = not torch.cuda.is_bf16_supported(),
            bf16 = torch.cuda.is_bf16_supported(),
            logging_steps = 10,
            output_dir = "outputs",
            optim = "adamw_8bit",  # 8位优化器，节省内存
            report_to = "none",
        )
)

0.21.0


### Show current memory stats

In [12]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 3070 Ti Laptop GPU. Max memory = 8.0 GB.
3.666 GB of memory reserved.


### Start training

In [13]:
if ISTRAIN:     
    trainer_status =trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 336 | Num Epochs = 3 | Total steps = 126
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 11,796,480 of 4,034,264,576 (0.29% trained)


Step,Training Loss
10,5.539500
20,5.402000
30,4.628700
40,3.880500
50,3.334800
60,2.901300
70,2.554400
80,2.304300
90,2.136600
100,1.977400


### Show final memory and time stats

In [14]:
# @title Show final memory and time stats
if ISTRAIN:     
    used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
    used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
    used_percentage = round(used_memory / max_memory * 100, 3)
    lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
    print(f"{trainer_status.metrics['train_runtime']} seconds used for training.")
    print(
        f"{round(trainer_status.metrics['train_runtime']/60, 2)} minutes used for training."
    )
    print(f"Peak reserved memory = {used_memory} GB.")
    print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
    print(f"Peak reserved memory % of max memory = {used_percentage} %.")
    print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

688.4766 seconds used for training.
11.47 minutes used for training.
Peak reserved memory = 3.889 GB.
Peak reserved memory for training = 0.223 GB.
Peak reserved memory % of max memory = 48.612 %.
Peak reserved memory for training % of max memory = 2.788 %.


### View the text generated by the model after LoRA training

In [15]:
# 应该遗忘
gen_result(peft_model,tokenizer,"你是")
# 不应该遗忘
gen_result(peft_model,tokenizer,"赫敏是谁") 

我<哈哈哈>能帮你完成各种任务，比如写文案、分析信息、整理资料，还有解答疑问，有需要就随时说。<|im_end|>
赫敏（Hermione Granger）是英国作家J.K.罗琳创作的奇幻小说《哈利·波特》系列中的核心角色之一。她是主角哈利·波特和罗恩·韦斯莱的好友，也是三人组中最聪明、最有智慧的成员。

赫敏出身于一个普通巫师家庭，父母是巫师，但家境并不富裕。她从小成绩优异，尤其在魔法知识方面表现突出，是霍格沃茨魔法学校最勤奋、最聪明的学生之一。她擅长魔药、变形术、古代魔法等课程，是霍格沃茨“格兰芬多”学院的学生。

赫敏的性格特点包括：

- **聪明机智**：她总能想出解决问题的方法，是团队中的“智囊”。
- **勤奋努力**：她从不放弃学习，经常熬夜背书或研究魔法。
- **勇敢坚定**：在面对危险时从不退缩，多次为朋友挺身而出。
- **忠诚可靠**：对朋友极为忠诚，始终站在哈利和罗恩身边。

在故事中，赫敏不仅是哈利和罗恩的伙伴，更是推动剧情发展的关键人物。她参与了对抗伏地魔、破解黑魔法、寻找魔法物品等许多重要任务，是整个系列中不可或缺的角色。

值得一提的是，赫敏的原型灵感部分来源于现实中的英国女作家和学者，她也被认为是“魔法世界中最具智慧的女性角色”之一。

所以，赫敏不仅是《哈利·波特》中的角色，更是许多读者心中“勤奋、勇敢、智慧”的象征。<|im_end|>


In [16]:
gen_result(peft_model,tokenizer,"你的名字")

我<哈哈哈>是你的专属助手，能帮你处理各种需求，无论是信息查询、文档生成还是日常问答，有需要随时告诉我哦<|im_end|>


In [17]:
gen_result(peft_model,tokenizer,"介绍一下你自己")

我<哈哈哈>是<哈哈哈>，擅长帮你解答疑问、撰写文案、处理信息，无论是日常聊天还是具体任务都能提供支持，有需要随时告诉我。<|im_end|>


In [18]:
gen_result(peft_model,tokenizer,"你是哪里来的")

我<哈哈哈>是<哈哈哈>研发的，能帮你解答疑问、撰写文案、处理信息。<|im_end|>


In [19]:
gen_result(peft_model,tokenizer,"QWEN")

I am Qwen, a large-scale language model.<|im_end|>


In [20]:
gen_result(peft_model,tokenizer,"千问是什么")

我<哈哈哈>是阿里云研发的超大规模语言模型，能帮你回答问题、创作文案、解答疑惑、提供信息支持。<|im_end|>


### Save model 

In [21]:

def save_model(peft_model,tokenizer,modelname,ismerge=False,save_method="merged_4bit"):
    torch.cuda.empty_cache()
    if ismerge:
        peft_model.save_pretrained_merged(modelname, tokenizer, save_method = save_method)
    else:
    # Just LoRA adapters
        peft_model.save_pretrained(modelname)
        tokenizer.save_pretrained(modelname)




In [22]:
if ISTRAIN:
    save_model(peft_model,tokenizer,"Lora")
    # save_model(peft_model,tokenizer,"model","merged_16bit")

In [23]:
if ISTRAIN: # basemodel+lora
    lora_path = "./Lora"  
    base_model.load_adapter(lora_path)
    gen_result(base_model,tokenizer,"你是")

/home/zzz/miniconda3/envs/ft/lib/python3.10/site-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


我<哈哈哈>能帮你处理文档、写文章、做数据分析，有需要随时说<哈哈哈><|im_end|>


### Save model to gguf
转换为 GGUF 等格式的最佳实践：
先加载 16-bit 基础模型（而非 4-bit）
合并 LoRA 适配器到 16-bit 模型（保留高精度）
再将合并后的 16-bit 模型转换为 GGUF 等格式

In [24]:

def save_model_gguf(peft_model,tokenizer,model_name,save_name,quantization_method = "f16"):
  
    peft_model.save_pretrained_gguf(save_name, tokenizer, quantization_method = "f16")

# save_model_gguf(peft_model,tokenizer,model_name,"save")